In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

import sys

if not sys.warnoptions:
    import warnings

    warnings.simplefilter("ignore")

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)


# ------------------------------------------------------------------------------------
def distance(x1, y1, x2, y2):
    return ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5


# ------------------------------------------------------------------------------------
def in_deep_matrix(p, y, x):
    # Возвращает новую матрицу меньшего размера, за вычитом строки и столбца
    return p.drop([y], axis=0).drop([x], axis=1)


# ------------------------------------------------------------------------------------
def reduction_matrix(p):
    # Производим редуцирование матрицы, возвращаем нижнюю границу
    bottom_line = 0

    # Находим минимум по каждой строке и вычитаем его
    for index, row in p.iterrows():
        min = row.min()
        if np.isinf(min):
            return np.inf
        bottom_line += min
        for key, value in row.items():
            p.at[index, key] -= min

    # Находим минимум по каждому столбцу и вычитаем его
    for key, value in p.items():
        min = value.min()
        if np.isinf(min):
            return np.inf
        bottom_line += min
        for index, row in value.items():
            p.at[index, key] -= min

    return bottom_line


# ------------------------------------------------------------------------------------
def partition_matrix(p):
    # Ищем элемент для разбиения матрицы на m1 и m2
    # Для этого производим оценку нулевых элементов матрицы
    max_sum = 0
    index_i = None
    index_j = None
    for index in p.index:
        for key in p.columns:
            if p[key][index] == 0:
                min_i = np.inf
                min_j = np.inf

                for k, v in p[key].items():  # по столбецу
                    if k != index and v < min_i:
                        min_i = v

                for k, v in p.loc[index].items():
                    if k != key and v < min_j:
                        min_j = v

                l = min_i + min_j
                if l > max_sum:
                    max_sum = l
                    index_i = index
                    index_j = key

    return [index_i, index_j, max_sum]


# ------------------------------------------------------------------------------------
def reverse_index(l, i, j):
    # Находим обрытный индекс для матрицы
    def in_dict(d, v):
        while v in d:
            v = d[v]
        return v

    ln = len(l)
    d1 = {l[k][0]: l[k][1] for k in range(0, ln, 1)}
    d2 = {l[k][1]: l[k][0] for k in range(0, ln, 1)}
    return [in_dict(d1, i), in_dict(d2, j)]


# ------------------------------------------------------------------------------------
def evaluation_matrix(p, res, bottom_line):
    # Оценка матрицы, поиск решения
    if len(p) == 1:
        res["steps"] += 1
        bottom_line += p.iat[0, 0]

        # Если текущее решение лучше, запоминаем его
        if bottom_line < res["global_min"]:
            res["global_min"] = bottom_line
            res["local_result"].append([p.index[0], p.columns[0]])
            res["best_result"] = res["local_result"].copy()
            print(
                "Решение лучше:", bottom_line, res["best_result"], "шаг: ", res["steps"]
            )
        return

    # Производим редуцирование матрицы, возвращаем минимальную нижнюю границу
    bottom_line += reduction_matrix(p)
    if np.isinf(bottom_line):
        return

    max_sum = 0
    while True:
        res["steps"] += 1
        # Находим элемент для разбиения на подмножества m1 и m2
        i, j, max_sum = partition_matrix(p)
        # Больше нет элементов для разбиения
        if i is None:
            return

        v_len = len(res["local_result"])
        # Рассматриваем m1 (соглашаемся на разбиение по элементу)
        if bottom_line < res["global_min"]:
            res["local_result"].append([i, j])
            p1 = in_deep_matrix(p, i, j)
            # Вычёркиваем обратный элемент только для матрицы большей чем 2х2, чтоб не получить inf
            if len(p1) > 2:
                i_reverse, j_reverse = reverse_index(res["local_result"], i, j)
                p1[j_reverse][i_reverse] = np.inf
            evaluation_matrix(p1, res, bottom_line)

        # Рассматриваем m2
        if res["global_min"] < bottom_line + max_sum:
            return
        p[j][i] = np.inf  # Исключаем не выбранный элемент
        res["local_result"] = res["local_result"][:v_len]  # Обрезаем список пути


# ------------------------------------------------------------------------------------
def return_res(res):
    l = res["best_result"]
    if l:
        d = {l[k][0]: l[k][1] for k in range(len(l))}
        li = []
        a = l[0][0]
        for v in range(len(l)):
            li.append(a)
            a = d[a]
        return li
    else:
        return []


n = 10
input_matrix = [
    [float("inf"), 92, 14, 71, 60, 20, 82, 86, 74, 74],
    [87, float("inf"), 23, 2, 21, 52, 1, 87, 29, 37],
    [1, 63, float("inf"), 20, 32, 75, 57, 21, 88, 48],
    [90, 58, 41, float("inf"), 59, 79, 14, 61, 61, 46],
    [61, 50, 54, 63, float("inf"), 100, 50, 6, 20, 72],
    [38, 17, 3, 88, 59, float("inf"), 8, 89, 52, 1],
    [83, 91, 59, 70, 43, 7, float("inf"), 34, 77, 80],
    [35, 49, 3, 1, 5, 53, 3, float("inf"), 92, 62],
    [17, 89, 43, 33, 73, 61, 99, 13, float("inf"), 47],
    [14, 71, 77, 86, 61, 39, 84, 79, 81, float("inf")],
]

f1 = pd.DataFrame(input_matrix)

f1

,0,1,2,3,4,5,6,7,8,9
0,inf,92.0,14.0,71.0,60.0,20.0,82.0,86.0,74.0,74.0
1,87.0,inf,23.0,2.0,21.0,52.0,1.0,87.0,29.0,37.0
2,1.0,63.0,inf,20.0,32.0,75.0,57.0,21.0,88.0,48.0
3,90.0,58.0,41.0,inf,59.0,79.0,14.0,61.0,61.0,46.0
4,61.0,50.0,54.0,63.0,inf,100.0,50.0,6.0,20.0,72.0
5,38.0,17.0,3.0,88.0,59.0,inf,8.0,89.0,52.0,1.0
6,83.0,91.0,59.0,70.0,43.0,7.0,inf,34.0,77.0,80.0
7,35.0,49.0,3.0,1.0,5.0,53.0,3.0,inf,92.0,62.0
8,17.0,89.0,43.0,33.0,73.0,61.0,99.0,13.0,inf,47.0
9,14.0,71.0,77.0,86.0,61.0,39.0,84.0,79.0,81.0,inf


In [32]:
# Инициализация массива решений
res = {"global_min": np.inf, "best_result": [], "local_result": [], "steps": 0}
# Запуск нахождения решения
evaluation_matrix(f1, res, 0)
df = pd.DataFrame()
df["результат"] = [res["global_min"]]
df["путь"] = [np.array(return_res(res)) + 1]
df["шагов"] = [res["steps"]]
df

Решение лучше: 161.0 [[6, 5], [0, 2], [9, 0], [5, 9], [8, 7], [4, 8], [2, 4], [7, 3], [1, 6], [np.int64(3), np.int64(1)]] шаг:  10


,результат,путь,шагов
0,161.0,"[7, 6, 10, 1, 3, 5, 9, 8, 4, 2]",62
